In [103]:
"""
Importing manim library

Creating Scenes for my Individual Project Video

"""

from manim import *
import numpy as np
from scipy.integrate import odeint, cumulative_trapezoid, solve_ivp
import math
import os

In [142]:
%%manim -qm -v WARNING OriginalSystemScene

class OriginalSystemScene(ThreeDScene): # Changed from Scene to ThreeDScene
    def construct(self):
        # TITLE SEQUENCE
        title = Tex(r"Paper Aeroplanes and Dynamical Systems", font_size=56)
        subtitle = Tex(r"Analysis of $\theta$ and $s$ over time", font_size=42, color=BLUE)
        subtitle.next_to(title, DOWN)
        title_group = VGroup(title, subtitle)
        title_group.shift(UP*2)
        
        self.play(Write(title_group), run_time=2)
        self.wait(1.5)
        self.play(Unwrite(title_group), run_time=1)
        self.wait(2)

        # Section 1
        Section1Title = Tex(r"1 Additional Force: Thrust", font_size=58)
        Section1Subtitle = Tex(r"Focusing on $D=0$", font_size=42, color=BLUE)
        Section1Subtitle.next_to(Section1Title, DOWN)
        Section1_group = VGroup(Section1Title, Section1Subtitle)
        Section1_group.shift(UP*2)

        self.play(Write(Section1_group), run_time=2)
        self.wait(1.5)
        self.play(Unwrite(Section1_group), run_time=1)
        self.wait(2)

        # --- 1. SOLVE SYSTEM ---
        def thrust_system(state, t):
            theta, s, y = state
            D, H, k, T = 1, 3, 1.5, 1.5
            s = max(s, 0.001) # Prevent division by zero
            d_theta = (s**2*np.exp(-y/H) - np.cos(theta))/s
            d_s = -np.sin(theta) - D*s**2*np.exp(-y/H) + T*np.exp(-k*y/H)
            d_y = s*np.sin(theta)
            return [d_theta, d_s, d_y]
        
        # Time setup
        t_min, t_max = 0, 15
        t = np.linspace(t_min, t_max, 1000)
        initial_state = [0, 1, 0]

        solution = odeint(thrust_system, initial_state, t)
        theta_vals = solution[:, 0]
        s_vals = solution[:, 1]
        y_vals = solution[:, 2]

        # Finding position of plane for graph
        v_x = s_vals*np.cos(theta_vals)
        x_vals = cumulative_trapezoid(v_x, t, initial=0)
        
        # --- 2. SHOW EQUATIONS ---
        eq1 = MathTex(r"\frac{d\theta}{dt}", r"=", r"\frac{s^2-\cos\theta}{s}")
        eq2 = MathTex(r"\frac{ds}{dt}", r"=", r"-\sin\theta-Ds^2")

        equations = VGroup(eq1, eq2)
        equations.arrange(DOWN, aligned_edge=LEFT, buff=0.2)

        self.play(Write(equations))
        
        # --- VISUALIZATION: Angle Theta and Speed S ---
        
        # 1. Setup the Diagram Elements
        diagram_center = RIGHT * 3.5 + UP * 0.5
        
        # A. Geometric Plane
        wing_top = Polygon(
            ORIGIN, RIGHT * 2, LEFT + UP * 0.8, LEFT * 0.2,
            color=GREEN, fill_opacity=1, stroke_width=1
        )
        wing_bot = Polygon(
            ORIGIN, RIGHT * 2, LEFT + DOWN * 0.8, LEFT * 0.2,
            color=GREEN_E, fill_opacity=1, stroke_width=1
        )
        plane_ref = VGroup(wing_top, wing_bot)
        plane_ref.move_to(ORIGIN).scale(0.25).move_to(diagram_center)
        
        # B. The Active Plane
        plane_icon = plane_ref.copy()
        
        # C. Reference Line
        ref_line = DashedLine(start=diagram_center, end=diagram_center + RIGHT * 3, color=GRAY)
        
        # D. Velocity Vector
        vec_arrow = Arrow(start=diagram_center, end=diagram_center + RIGHT * 1.5, buff=0, color=RED)
        
        # E. Labels
        label_s = MathTex("s", color=RED).next_to(vec_arrow, UP)
        
        # Create the label but make it INVISIBLE initially
        label_theta = MathTex(r"\theta", color=YELLOW)
        label_theta.set_opacity(0) 
        
        # F. Angle Arc
        arc_theta = VMobject(color=YELLOW)
        arc_theta.set_opacity(0)
        
        # Group for easier handling
        diagram = VGroup(ref_line, vec_arrow, plane_icon, label_s, arc_theta, label_theta)
        
        self.play(FadeIn(diagram))
        
        # 2. Setup Trackers
        theta_tracker = ValueTracker(0) 
        s_tracker = ValueTracker(1.5)
        
        # 3. Define Update Function
        def update_diagram(mob):
            deg_value = theta_tracker.get_value()
            angle_rad = deg_value * DEGREES
            length = s_tracker.get_value()
            
            # Update Plane
            plane_icon.become(plane_ref).rotate(angle_rad)
            
            # Update Vector
            new_vec_end = diagram_center + np.array([length * np.cos(angle_rad), length * np.sin(angle_rad), 0])
            vec_arrow.put_start_and_end_on(diagram_center, new_vec_end)
            
            # Update 's' Label
            label_s.move_to(vec_arrow.get_center() + UP * 0.3 + LEFT * 0.1)
            
            # Update Theta Arc & Label
            if abs(deg_value) > 1:
                arc_theta.set_opacity(1)
                label_theta.set_opacity(1)
                
                # Draw geometric arc
                new_arc = Arc(
                    radius=0.8, 
                    start_angle=0, 
                    angle=angle_rad, 
                    arc_center=diagram_center,
                    color=YELLOW
                )
                arc_theta.set_points(new_arc.get_points())
                
                # Position Label
                half_angle = angle_rad / 2
                label_x = diagram_center[0] + 1.3 * np.cos(half_angle)
                label_y = diagram_center[1] + 1.3 * np.sin(half_angle)
                label_theta.move_to([label_x, label_y, 0])
            else:
                arc_theta.set_opacity(0)
                label_theta.set_opacity(0)

        # 4. Animate
        diagram.add_updater(update_diagram)
        
        self.play(theta_tracker.animate.set_value(35), run_time=2)
        self.wait(0.5)
        self.play(s_tracker.animate.set_value(2.5), run_time=2)
        self.wait(0.5)
        self.play(theta_tracker.animate.set_value(-25), run_time=2)
        self.wait(1)
        
        # Cleanup
        diagram.remove_updater(update_diagram)
        self.play(FadeOut(diagram))

        # --- TRANSITION TO NEW EQUATIONS ---
        eq3 = MathTex(r"\frac{ds}{dt}", r"=", r"-\sin\theta-Ds^2", r"+", r"T_{thrust}")
        eq3[4].set_color(RED)

        # Alignment Fix
        eq3.move_to(eq2.get_center())
        shift_vector = eq2[1].get_center() - eq3[1].get_center()
        eq3.shift(shift_vector)
        
        self.play(
            TransformMatchingTex(eq2, eq3, path_arc=90 * DEGREES),
            run_time=1.5
        )

        equations = VGroup(eq1, eq3)
        self.wait(3)

        # --- DEFINE NEW EQUATIONS ---
        eq4 = MathTex(r"\frac{dy}{dt}", r"=", r"s\sin\theta")
        
        eq5 = MathTex(r"\frac{d\theta}{dt}", r"=", r"s", r"e^{-y/H}", r"-", r"\frac{\cos\theta}{s}")
        
        eq6 = MathTex(
            r"\frac{ds}{dt}",    # 0
            r"=",                # 1
            r"-\sin\theta",      # 2 
            r"-Ds^2",            # 3 
            r"e^{-y/H}",         # 4 
            r"+",                # 5
            r"T_{thrust}",       # 6 
            r"e^{-ky/H}"         # 7 
        )

        eq5[3].set_color(BLUE)
        eq6[4].set_color(BLUE)
        eq6[7].set_color(BLUE)

        # Alignment Logic
        eq5.move_to(eq1.get_center())
        eq5.shift(eq1[1].get_center() - eq5[1].get_center())

        eq6.move_to(eq3.get_center())
        eq6.shift(eq3[1].get_center() - eq6[1].get_center())

        eq4.next_to(eq6, DOWN, buff=0.2, aligned_edge=LEFT)
        eq4.shift(RIGHT * (eq6[1].get_center()[0] - eq4[1].get_center()[0]))

        # Animation
        self.play(
            TransformMatchingTex(eq1, eq5), 
            TransformMatchingTex(eq3, eq6, key_map={r"T_{thrust}": r"T_{thrust}"}),
            run_time=1.5
        )
        
        equations = VGroup(eq4, eq5, eq6)
        self.wait(2)
        
        self.play(Write(eq4), run_time=1.5)
        self.play(equations.animate.move_to(ORIGIN), run_time=1.5)

        self.play(equations.animate.set_color(WHITE), run_time=2)
        self.wait(1)

        # --- BRACES ---
        brace = Brace(eq5[2:4], UP)
        label = brace.get_text("Lift").scale(1)
        self.play(GrowFromCenter(brace), FadeIn(label))
        self.wait(2)
        self.play(FadeOut(brace), FadeOut(label), run_time=0.5)

        brace = Brace(eq5[4:6], UP)
        brace1 = Brace(eq6[2], UP)
        label = brace.get_text("Gravity Components").scale(1)
        self.play(GrowFromCenter(brace), GrowFromCenter(brace1), FadeIn(label))
        self.wait(2)
        self.play(FadeOut(brace), FadeOut(brace1), FadeOut(label), run_time=0.5)

        brace = Brace(eq6[3:5], DOWN)
        label = brace.get_text("Drag").scale(1)
        self.play(GrowFromCenter(brace), FadeIn(label))
        self.wait(2)
        self.play(FadeOut(brace), FadeOut(label), run_time=0.5)

        brace = Brace(eq6[6:], UP)
        label = brace.get_text("Thrust").scale(1)
        self.play(GrowFromCenter(brace), FadeIn(label))
        self.wait(2)
        self.play(FadeOut(brace), FadeOut(label), run_time=0.5)

        self.wait(1)
        self.play(equations.animate.scale(0.45).to_corner(UR, buff=0.5))
        self.wait(4)

        # --- 3. PLOT GRAPH ---
        axes = Axes(
            x_range=[t_min, t_max, 1],
            y_range=[min(min(theta_vals), min(s_vals), min(y_vals))-0.5, max(max(theta_vals), max(s_vals), max(y_vals))+0.5, 0.5],
            x_length=8,
            y_length=5,
            axis_config={"color": BLUE},
            x_axis_config={"numbers_to_include": np.arange(0, t_max + 1, 2)},
            y_axis_config={"numbers_to_include": np.arange(round(min(min(theta_vals), min(s_vals), min(y_vals))*2)/2-0.5, round(max(max(theta_vals), max(s_vals), max(y_vals))*2)/2+1, 0.5)},
            tips=False,
        )
        axes.to_edge(LEFT, buff=1).shift(DOWN*0.5)
        labels = axes.get_axis_labels(x_label="Time (t)", y_label="Value")

        curve_theta = VMobject().set_points_as_corners([axes.c2p(time, val) for time, val in zip(t, theta_vals)]).set_color(YELLOW).set_stroke(width=3)
        curve_s = VMobject().set_points_as_corners([axes.c2p(time, val) for time, val in zip(t, s_vals)]).set_color(RED).set_stroke(width=3)
        curve_y = VMobject().set_points_as_corners([axes.c2p(time, val) for time, val in zip(t, y_vals)]).set_color(BLUE).set_stroke(width=3)

        legend = VGroup(
            VGroup(Line(color=YELLOW), MathTex(r"\theta", color=YELLOW)).arrange(RIGHT),
            VGroup(Line(color=RED), MathTex(r"s", color=RED)).arrange(RIGHT),
            VGroup(Line(color=BLUE), MathTex(r"y", color=BLUE)).arrange(RIGHT)
        ).arrange(DOWN, aligned_edge=LEFT).next_to(axes, UP)

        self.play(Write(axes), Write(labels), FadeIn(legend))
        self.play(Create(curve_theta), Create(curve_s), Create(curve_y), run_time=5, rate_func=linear)
        self.wait(3)

        # Move Graph
        graph_group = VGroup(axes, labels, curve_theta, curve_s, curve_y, legend)
        box = SurroundingRectangle(graph_group, color=WHITE, buff=0.2)
        graph_group.add(box)
        
        self.play(Create(box))
        self.play(graph_group.animate.scale(0.3).to_corner(UL, buff=0.5), run_time=1.5)
        self.wait(3)

        # --- TRAJECTORY GRAPH ---
        x_min, x_max = min(x_vals), max(x_vals)
        y_min, y_max = min(y_vals), max(y_vals)

        traj_axes = Axes(
            x_range=[x_min, x_max], y_range=[y_min-0.5, y_max+0.5],
            x_length=11, y_length=4.5,
            axis_config={"color":BLUE, "include_tip":False}
        ).add_coordinates().to_edge(DOWN, buff=0.5)

        traj_label = Tex("Flight Trajectory $(x,y)$").next_to(traj_axes, UP)
        flight_path = VMobject().set_points_as_corners([traj_axes.c2p(x, y) for x, y in zip(x_vals, y_vals)]).set_color(GREEN).set_stroke(width=3)
        
        self.play(Write(traj_axes), Write(traj_label))
        self.play(Create(flight_path), run_time=6, rate_func=linear)
        self.wait(4)

        self.play(FadeOut(box), run_time=1)
        graph_group.remove(box)

        # Split Screen
        traj_group = VGroup(traj_axes, traj_label, flight_path)
        self.play(Unwrite(equations), run_time=1)
        self.play(
            graph_group.animate.scale(2.25).move_to(LEFT*3.4),
            traj_group.animate.scale(0.55).move_to(RIGHT*3.5),
            run_time=2
        )
        self.wait(4)

        # --- TRACING WITH LIVE COUNTERS ---
        
        # 1. Tracers
        c_theta = Circle(radius=0.15, color=YELLOW, stroke_width=2).move_to(curve_theta.get_start())
        c_s = Circle(radius=0.15, color=RED, stroke_width=2).move_to(curve_s.get_start())
        c_y = Circle(radius=0.15, color=BLUE, stroke_width=2).move_to(curve_y.get_start())

        # 2. Geometric Plane (Trajectory)
        wing_top_t = Polygon(ORIGIN, RIGHT * 2, LEFT + UP * 0.8, LEFT * 0.2, color=GREEN, fill_opacity=1, stroke_width=1)
        wing_bot_t = Polygon(ORIGIN, RIGHT * 2, LEFT + DOWN * 0.8, LEFT * 0.2, color=GREEN_E, fill_opacity=1, stroke_width=1)
        c_flight = VGroup(wing_top_t, wing_bot_t).scale(0.15).move_to(flight_path.get_start())
        c_flight_ref = c_flight.copy()

        # 3. Counters
        theta_counter = DecimalNumber(0, num_decimal_places=2, color=YELLOW).next_to(c_theta, UP)
        s_counter = DecimalNumber(0, num_decimal_places=2, color=RED).next_to(c_s, UP)
        y_counter = DecimalNumber(0, num_decimal_places=2, color=BLUE).next_to(c_y, UP)

        traj_x_num = DecimalNumber(0, num_decimal_places=2, color=GREEN)
        traj_y_num = DecimalNumber(0, num_decimal_places=2, color=GREEN)
        traj_label = VGroup(
            MathTex("(", color=GREEN), traj_x_num, MathTex(",", color=GREEN), traj_y_num, MathTex(")", color=GREEN)
        ).arrange(RIGHT, buff=0.1).next_to(c_flight, UP)

        self.play(
            FadeIn(c_theta), FadeIn(c_s), FadeIn(c_y), FadeIn(c_flight),
            FadeIn(theta_counter), FadeIn(s_counter), FadeIn(y_counter), FadeIn(traj_label)
        )

        # 4. Updater
        alpha_tracker = ValueTracker(0)

        def update_trace(mob):
            alpha = alpha_tracker.get_value()
            idx = int(alpha * (len(t) - 1))
            
            # Data
            current_time = t[idx]
            current_vals = [theta_vals[idx], s_vals[idx], y_vals[idx]]
            current_pos = [x_vals[idx], y_vals[idx]]
            
            # Position Time Markers
            c_theta.move_to(axes.c2p(current_time, current_vals[0]))
            c_s.move_to(axes.c2p(current_time, current_vals[1]))
            c_y.move_to(axes.c2p(current_time, current_vals[2]))
            
            # Manual Rotation Logic for Trajectory
            p0 = traj_axes.c2p(current_pos[0], current_pos[1])
            
            dx = np.cos(current_vals[0])
            dy = np.sin(current_vals[0])
            p1 = traj_axes.c2p(current_pos[0] + dx, current_pos[1] + dy)
            
            screen_dx = p1[0] - p0[0]
            screen_dy = p1[1] - p0[1]
            visual_angle = np.arctan2(screen_dy, screen_dx)
            
            c_flight.become(c_flight_ref)
            c_flight.move_to(p0)
            c_flight.rotate(visual_angle)

            # Update Labels
            theta_counter.set_value(current_vals[0]).next_to(c_theta, UP, buff=0.2)
            s_counter.set_value(current_vals[1]).next_to(c_s, UP, buff=0.2)
            y_counter.set_value(current_vals[2]).next_to(c_y, UP, buff=0.2)
            
            traj_x_num.set_value(current_pos[0])
            traj_y_num.set_value(current_pos[1])
            traj_label.arrange(RIGHT, buff=0.1).next_to(c_flight, UP, buff=0.2)

        c_theta.add_updater(update_trace)
        self.play(alpha_tracker.animate.set_value(1), run_time=13, rate_func=linear)
        c_theta.remove_updater(update_trace)
        
        self.wait(2)
        
        # --- PHASE CYLINDER TRANSITION (s = Radius, y = Height) ---
        self.play(FadeOut(Group(*self.mobjects)))
        
        # 1. SETUP POLAR AXES
        # z_range needs to cover the max height (y) reached in simulation
        y_max_val = max(y_vals)
        polar_axes = ThreeDAxes(
            x_range=[-4, 4, 1], 
            y_range=[-4, 4, 1], 
            z_range=[0, y_max_val + 1, 2], 
            z_length=6
        )
        
        # 2. CREATE POLAR GRID ("Floor")
        polar_grid = VGroup()
        
        # Concentric Circles (Visual guide for Speed s)
        # s usually varies between 1 and 3 in your simulation
        for r in [1, 2, 3]: 
            circle = Circle(radius=r, color=BLUE_E, stroke_opacity=0.5)
            polar_grid.add(circle)
            label = MathTex(f"s={r}", color=BLUE_C, font_size=20).move_to(RIGHT * r + UP * 0.1)
            polar_grid.add(label)
            
        # Radial Lines (Theta)
        for angle in np.arange(0, 360, 45):
            line = Line(ORIGIN, RIGHT * 3.5, color=BLUE_E, stroke_opacity=0.5)
            line.rotate(angle * DEGREES, about_point=ORIGIN)
            polar_grid.add(line)
            
        # 3. ADD ANGULAR LABELS (Theta)
        theta_labels = VGroup()
        angles_tex = {0: "0", 90: r"\pi/2", 180: r"\pi", 270: r"-pi/2"}
        
        for angle_deg, tex in angles_tex.items():
            rad = angle_deg * DEGREES
            pos = np.array([3.8 * np.cos(rad), 3.8 * np.sin(rad), 0])
            label = MathTex(tex, color=YELLOW).scale(0.7).move_to(pos)
            label.rotate(90 * DEGREES, axis=RIGHT) 
            label.rotate(angle_deg * DEGREES + 90*DEGREES, axis=OUT)
            theta_labels.add(label)

        # 4. GENERATE 3D TRAJECTORY
        # Mapping: 
        #   Angle  -> Theta (simulation)
        #   Radius -> s     (simulation)
        #   Height -> y     (simulation)
        points_3d = []
        for th, s, alt in zip(theta_vals, s_vals, y_vals):
            # Polar conversion: x = s*cos(theta), y = s*sin(theta)
            # We scale the coordinates slightly if needed to fit the screen
            # But physically, x = s*cos(th), y = s*sin(th), z = alt
            
            # NOTE: axes.c2p handles the visual scaling relative to the axis ranges defined above
            points_3d.append(polar_axes.c2p(s * np.cos(th), s * np.sin(th), alt))

        cyl_traj = VMobject()
        cyl_traj.set_points_as_corners(points_3d).set_color(YELLOW).set_stroke(width=3)
        
        # Title
        phase_title = Tex(r"Phase Cylinder: $s$(radius), $\theta$(angle), $y$(height)").to_edge(UP)
        self.add_fixed_in_frame_mobjects(phase_title)
        self.play(Write(phase_title))

        # 5. ANIMATE
        self.move_camera(phi=65 * DEGREES, theta=330 * DEGREES, run_time=2)
        
        self.play(
            Create(polar_axes.z_axis),
            Write(polar_axes.get_z_axis_label(MathTex("y"))), # Z axis represents Height (y)
            Create(polar_grid),
            Write(theta_labels),
            run_time=2
        )
        
        # Trace Path
        tracer_dot = Sphere(radius=0.1, color=GREEN).move_to(points_3d[0])
        
        # Create a "Ghost" projection on the floor (Optional, looks cool)
        # Helps see the relationship between s and theta
        # floor_traj = cyl_traj.copy().set_z(0).set_color(GRAY).set_stroke(opacity=0.5)
        # self.add(floor_traj)

        self.begin_ambient_camera_rotation(rate=0.15)
        
        self.play(
            Create(cyl_traj),
            MoveAlongPath(tracer_dot, cyl_traj),
            run_time=10,
            rate_func=linear
        )
        
        self.wait(3)
        self.stop_ambient_camera_rotation()

        # "Unwrite" animation style
        self.play(FadeOut(Group(*self.mobjects)), run_time=2)

Manim Community v0.18.1

Animation 10: _MethodAnimation(ValueTracker):   0%|          | 0/1 [00:00<?, ?it/s] <string>:136: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.


In [141]:
%%manim -qm -v WARNING UpdatedSystemScene

from manim import *
import numpy as np
from scipy.integrate import odeint, cumulative_trapezoid
import math

class UpdatedSystemScene(Scene):
    def construct(self):
        # ==========================================
        # PART 1: EQUATIONS
        # ==========================================

        Section2Title = Tex(r"2 Even Faster: Supersonic Flight", font_size=58)
        Section2Subtitle = Tex(r"Focusing on $D=0$", font_size=42, color=BLUE)
        Section2Subtitle.next_to(Section2Title, DOWN)
        Section2_group = VGroup(Section2Title, Section2Subtitle)
        Section2_group.shift(UP*2)

        self.play(Write(Section2_group), run_time=2)
        self.wait(1.5)
        self.play(Unwrite(Section2_group), run_time=1)
        self.wait(0.5)

        eq1 = MathTex(r"\frac{d\theta}{dt}", r"=", r"se^{-y/H}", r"-\frac{\cos\theta}{s}")
        eq2 = MathTex(r"\frac{ds}{dt}", r"=", r"-\sin\theta", r"-D", r"s^2e^{-y/H}", r"+", r"Te^{-ky/H}")
        eq3 = MathTex(r"\frac{dy}{dt}", r"=", r"s\sin\theta")
        equations_original = VGroup(eq1, eq2, eq3).arrange(DOWN, aligned_edge=LEFT, buff=0.5)
        
        self.play(Write(equations_original), run_time=2)
        self.wait(2)

        eq4 = MathTex(
            r"\frac{d\theta}{dt}", r"=", 
            r"\frac{4(\alpha-\theta)se^{-y/H}}{\sqrt{(s/s_v)^2-1}}", 
            r"-\frac{\cos\theta}{s}"
        )
        eq5 = MathTex(
            r"\frac{ds}{dt}", r"=", r"-\sin\theta", 
            r"-", 
            r"\left( D_{sub} + \frac{D_{super}-D_{sub}}{1+e^{-b(s-s_v)}} \right)", 
            r"s^2e^{-y/H}", 
            r"+", 
            r"Te^{-ky/H}"
        )
        eq3_copy = eq3.copy()
        equations_new = VGroup(eq4, eq5, eq3_copy).arrange(DOWN, aligned_edge=LEFT, buff=0.6)
        
        if equations_new.width > config.frame_width * 0.9:
            equations_new.scale_to_fit_width(config.frame_width * 0.9)
        equations_new.move_to(ORIGIN)

        self.play(
            TransformMatchingTex(eq1, eq4),
            TransformMatchingTex(eq2, eq5),
            TransformMatchingTex(eq3, eq3_copy),
            run_time=3
        )
        self.wait(2)
        
        brace = Brace(eq5[4], DOWN)
        label = brace.get_text("$D(s)$").scale(1)
        self.play(GrowFromCenter(brace), FadeIn(label))
        self.wait(2)
        self.play(FadeOut(brace), FadeOut(label), run_time=0.5)
        
        self.play(equations_new.animate.scale(0.45).to_corner(UR, buff=0.5))
        self.wait(1)

        # ==========================================
        # PART 2: PYTHON SOLVER
        # ==========================================

        def plane_dynamics(t, v, s_v, D_sub, D_super, k_drag, T_base, H, k_thrust, pitch_attitude):
            theta, s, y = v[0], v[1], v[2]
            D_current = D_sub + (D_super - D_sub) / (1 + np.exp(-k_drag * (s - s_v)))
            Thrust = T_base * np.exp(-k_thrust * y / H)
            
            if s <= s_v:
                d_theta = (s**2 * np.exp(-y/H) - np.cos(theta)) / s
                d_s = -np.sin(theta) - D_sub * (s**2) * np.exp(-y/H) + Thrust
            else:
                M = s / s_v
                term = np.sqrt(M**2 - 1)
                if term < 0.1: term = 0.1
                alpha = pitch_attitude - theta
                d_theta = ((4 * alpha) / term) * s * np.exp(-y/H) - (np.cos(theta) / s)
                d_s = -np.sin(theta) - D_current * (s**2) * np.exp(-y/H) + Thrust
            
            d_y = s * np.sin(theta)
            return [d_theta, d_s, d_y]

        s_v, D_sub, D_super, k_drag = 3.0, 1, 2, 5
        T_base, H, k_thrust, pitch_attitude = 25.0, 3, 1.5, 0.03
        args = (s_v, D_sub, D_super, k_drag, T_base, H, k_thrust, pitch_attitude)

        t_max_sim = 15
        t = np.linspace(0, t_max_sim, 400) 
        initial_state = [0, 1.5, 0] 

        def odeint_wrapper(v, t, *args):
            return plane_dynamics(t, v, *args)

        solution = odeint(odeint_wrapper, initial_state, t, args=args)
        theta_vals, s_vals, y_vals = solution[:, 0], solution[:, 1], solution[:, 2]
        v_x = s_vals * np.cos(theta_vals)
        x_vals = cumulative_trapezoid(v_x, t, initial=0)

        # ==========================================
        # PART 3: FIRST PLOTS
        # ==========================================

        def round_up_to_multiple(val, multiple):
            return math.ceil(val / multiple) * multiple

        time_range = [0, 15, 5]
        max_x_val = max(5, round_up_to_multiple(np.max(x_vals), 5)) 
        dist_range = [0, max_x_val, 5]
        max_s_val = max(1, math.ceil(np.max(s_vals)))
        speed_range = [0, max_s_val, 1]
        max_y_val = max(1, math.ceil(np.max(y_vals)))
        alt_range = [0, max_y_val, 1]

        ax_traj = Axes(x_range=dist_range, y_range=alt_range, x_length=5, y_length=3, axis_config={"font_size": 20})
        ax_traj.to_edge(DL, buff=1.0).add_coordinates()
        group_traj = VGroup(ax_traj, ax_traj.get_axis_labels("x", "y"), Tex("Trajectory", font_size=30).next_to(ax_traj, UP))

        ax_speed = Axes(x_range=time_range, y_range=speed_range, x_length=5, y_length=3, axis_config={"font_size": 20})
        ax_speed.next_to(ax_traj, RIGHT, buff=1.5).add_coordinates()
        group_speed = VGroup(ax_speed, ax_speed.get_axis_labels("t", "s"), Tex("Speed Profile", font_size=30).next_to(ax_speed, UP))

        traj_curve = VMobject(color=YELLOW).set_points_smoothly([ax_traj.c2p(x, y) for x, y in zip(x_vals, y_vals)])
        speed_curve = VMobject(color=RED).set_points_smoothly([ax_speed.c2p(ti, si) for ti, si in zip(t, s_vals)])
        sv_line = ax_speed.get_horizontal_line(ax_speed.c2p(t_max_sim, s_v), color=BLUE)
        sv_label = Tex(r"$s_v$", color=BLUE, font_size=24).next_to(sv_line, LEFT)

        self.play(Write(group_traj), Write(group_speed), run_time=2)
        self.play(Create(sv_line), Write(sv_label), run_time=1)
        self.play(Create(traj_curve), Create(speed_curve), run_time=4, rate_func=linear)
        self.wait(2)

        # ==========================================
        # PART 4: MOVE GRAPHS UP
        # ==========================================
        
        all_plots = VGroup(group_traj, traj_curve, group_speed, speed_curve, sv_line, sv_label)
        self.play(all_plots.animate.scale(0.55).to_corner(UL, buff=0.5), run_time=2)
        self.wait(1)

        # ==========================================
        # PART 5: JACOBIAN MATRIX
        # ==========================================

        jacobian_tex = MathTex(
            r"J = \begin{pmatrix}"
            r"\frac{-4se^{-y/H}}{\sqrt{(s/s_v)^2-1}} + \frac{\sin\theta}{s} & "
            r"\frac{-4(\alpha-\theta)se^{-y/H}}{((s/s_v)^2-1)^{3/2}} + \frac{\cos\theta}{s^2} & "
            r"\frac{4(\alpha-\theta)se^{-y/H}}{H\sqrt{(s/s_v)^2-1}} \\"
            r"-\cos\theta & "
            r"-(2sD(s) +s^2D'(s))e^{-y/H} & "
            r"\frac{1}{H}(D(s)s^2e^{-y/H} - kT_{y=0}e^{-y/H}) \\"
            r"s\cos\theta & "
            r"\sin\theta & "
            r"0"
            r"\end{pmatrix}",
            font_size=24 
        )
        jacobian_tex.move_to(ORIGIN).shift(DOWN * 0.5)

        stability_text = Tex(
            r"If $\max(Re(\lambda_i)) < 0$ $\implies$ plane is stable.",
            font_size=32
        ).next_to(jacobian_tex, DOWN, buff=0.5)

        self.play(Write(jacobian_tex), Write(stability_text), run_time=2)
        self.wait(3)
        self.play(Unwrite(jacobian_tex), Unwrite(stability_text), run_time=1)

        # ==========================================
        # PART 6: IMPORT LAMBDA FROM CSV (FIXED)
        # ==========================================
        
        try:
            # We skip the first row (header) just in case the CSV has one
            # filling_values fills missing/NaN spots with 0
            csv_data = np.genfromtxt('lambda_data.csv', delimiter=',', skip_header=0, filling_values=0)
            
            # If the CSV import results in 1D array of NaNs (e.g. text file read incorrectly)
            # This line converts any remaining NaNs to 0.0
            csv_data = np.nan_to_num(csv_data)

            # Check if csv length matches time steps.
            if len(csv_data) != len(t):
                x_old = np.linspace(0, t_max_sim, len(csv_data))
                lambda_vals = np.interp(t, x_old, csv_data)
            else:
                lambda_vals = csv_data
                
        except (OSError, ValueError):
            print("WARNING: Error reading 'lambda_data.csv'. Using dummy data.")
            # Fallback data to prevent crash
            lambda_vals = np.linspace(-0.5, -0.1, len(t))

        # Final safety check: Convert any stubborn NaNs to 0
        lambda_vals = np.nan_to_num(lambda_vals, nan=0.0)

        # ==========================================
        # PART 7: SECOND PLOTS (BIGGER & BOTTOM)
        # ==========================================

        

        max_theta = max(0.1, math.ceil(np.max(theta_vals)*10)/10)
        min_theta = min(-0.1, math.floor(np.min(theta_vals)*10)/10)
        theta_range = [min_theta, max_theta, 0.1]
        
        # Calculate Lambda Range safely
        min_l_val = np.min(lambda_vals)
        max_l_val = np.max(lambda_vals)
        
        # Determine Integer Steps
        min_lam = -0.01
        max_lam = 0.01
        
        # Force a valid range even if data is flat 0
        if max_lam <= min_lam:
            max_lam = min_lam + 0.5
            
        # Ensure 0 is included for stability line visualization
        if max_lam < 0.1: max_lam = 0.1
        if min_lam > -0.1: min_lam = -0.1
        
        lambda_range = [min_lam, max_lam, 0.025]

        ax_theta = Axes(x_range=time_range, y_range=theta_range, x_length=5, y_length=3, axis_config={"font_size": 14})
        ax_theta.to_edge(DL, buff=1.0).add_coordinates()
        group_theta = VGroup(ax_theta, ax_theta.get_axis_labels("t", r"\theta"), Tex(r"Angle $\theta$", font_size=24).next_to(ax_theta, UP))

        ax_lambda = Axes(x_range=time_range, y_range=lambda_range, x_length=5, y_length=3, axis_config={"font_size": 14})
        ax_lambda.next_to(ax_theta, RIGHT, buff=1.5).add_coordinates()
        group_lambda = VGroup(ax_lambda, ax_lambda.get_axis_labels("t", r"max(Re(\lambda))"), Tex(r"Stability", font_size=24).next_to(ax_lambda, UP).shift(UP*0.5))

        theta_curve = VMobject(color=GREEN).set_points_smoothly([ax_theta.c2p(ti, thi) for ti, thi in zip(t, theta_vals)])
        lambda_curve = VMobject(color=ORANGE).set_points_smoothly([ax_lambda.c2p(ti, li) for ti, li in zip(t, lambda_vals)])
        stab_line = ax_lambda.get_horizontal_line(ax_lambda.c2p(t_max_sim, 0), color=RED)

        group_theta_all = VGroup(group_theta, theta_curve)
        group_lambda_all = VGroup(group_lambda, lambda_curve, stab_line)
        
        # Note: We do NOT move "all_plots" here. It stays at UL from Part 4.
        
        self.play(Write(group_theta), Write(group_lambda), run_time=2)
        self.play(Create(stab_line))
        self.play(Create(theta_curve), Create(lambda_curve), run_time=4, rate_func=linear)
        self.wait(5)

        self.play(FadeOut(Group(*self.mobjects)), run_time=2)

        # ==========================================
        # PART 8: END SEQUENCE
        # ==========================================

        EndTitle = Tex(r"Conclusion", font_size=64)
        EndSubtitle = Tex(r"Individual Report", font_size=50, color=BLUE)
        EndSubtitle.next_to(EndTitle, DOWN)
        End_group = VGroup(EndTitle, EndSubtitle)
        End_group.to_edge(UP, buff=0.1)
        End_group.shift(DOWN*0.1)

        self.play(Write(End_group), run_time=2)
        self.wait(1.5)

        original_eq1 = MathTex(r"\frac{d\theta}{dt}", r"=", r"\frac{s^2-\cos\theta}{s}", font_size=32)
        original_eq2 = MathTex(r"\frac{ds}{dt}", r"=", r"-\sin\theta", r"-Ds^2", font_size=32)
        original_subtitle = Tex(r"Original Model", font_size=34, color=RED)
        original_system_group = VGroup(original_subtitle, original_eq1, original_eq2).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
        original_system_group.to_edge(LEFT, buff=0.1)
        original_system_group.shift(UP*0.3)

        thrust_blist = [
                    "Varying thrust and drag with altitude",
                    "Existence of an equilibrium",
                    "Phase cylinders"
                ]
        thrust_bullet_group = VGroup()
        for item in thrust_blist:
            # Create the dot and text
            dot = Dot(color=BLUE)
            text = Tex(item, font_size=24)
            # Position text next to dot
            text.next_to(dot, RIGHT, buff=0.2)
            # Group them together
            line = VGroup(dot, text)
            thrust_bullet_group.add(line)
        thrust_bullet_group.arrange(DOWN, aligned_edge=LEFT, buff=0.2)

        thrust_eq1 = MathTex(r"\frac{d\theta}{dt}", r"=", r"\frac{s^2e^{-y/H}-\cos\theta}{s}", font_size=28)
        thrust_eq2 = MathTex(r"\frac{ds}{dt}", r"=", r"-\sin\theta", r"-", r"Ds^2e^{-y/H}", r"+", r"T_{thrust}e^{-ky/H}", font_size=24)
        thrust_eq3 = MathTex(r"\frac{dy}{dt}", r"=", r"s\sin\theta", font_size=28)
        thrust_subtitle = Tex(r"Additional Force: Thrust", font_size=34, color=YELLOW)
        thrust_system_group = VGroup(thrust_subtitle, thrust_eq1, thrust_eq2, thrust_eq3, thrust_bullet_group).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
        thrust_system_group.shift(UP*0.3, LEFT*1.1)

        supersonic_blist = [
                    r"Ackeret's Theory",
                    r"Stability",
                    r"Equilibrium at $\alpha\approx 0.012$"
                ]
        supersonic_bullet_group = VGroup()
        for item in supersonic_blist:
            # Create the dot and text
            dot = Dot(color=BLUE)
            text = Tex(item, font_size=24)
            # Position text next to dot
            text.next_to(dot, RIGHT, buff=0.2)
            # Group them together
            line = VGroup(dot, text)
            supersonic_bullet_group.add(line)
        supersonic_bullet_group.arrange(DOWN, aligned_edge=LEFT, buff=0.2)

        supersonic_eq1 = MathTex(
            r"\frac{d\theta}{dt}", r"=", 
            r"\frac{4(\alpha-\theta)se^{-y/H}}{\sqrt{(s/s_v)^2-1}}", 
            r"-\frac{\cos\theta}{s}", font_size=24
        )
        supersonic_eq2 = MathTex(
            r"\frac{ds}{dt}", r"=", r"-\sin\theta", 
            r"-", 
            r"\left( D_{sub} + \frac{D_{super}-D_{sub}}{1+e^{-b(s-s_v)}} \right)", 
            r"s^2e^{-y/H}", 
            r"+", 
            r"Te^{-ky/H}", font_size=18
        )
        supersonic_eq3 = MathTex(r"\frac{dy}{dt}", r"=", r"s\sin\theta", font_size=26)
        supersonic_subtitle = Tex(r"Even Faster: Supersonic Flight", font_size=34, color=GREEN)
        supersonic_system_group = VGroup(supersonic_subtitle, supersonic_eq1, supersonic_eq2, supersonic_eq3, supersonic_bullet_group).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
        supersonic_system_group.to_edge(RIGHT, buff=0.1)
        supersonic_system_group.shift(UP*0.3)

        self.play(Write(original_subtitle), run_time=1)
        self.wait(1)
        self.play(Write(original_eq1), Write(original_eq2), run_time=2)
        self.wait(3)

        arrow1 = Arrow(
            start=original_system_group.get_right(),
            end=thrust_system_group.get_left(),
            buff=0.1
        )
        self.play(GrowArrow(arrow1), run_time=1)
        self.wait(1)

        self.play(Write(thrust_subtitle), run_time=1)
        self.wait(1)
        self.play(Write(thrust_eq1), Write(thrust_eq2), Write(thrust_eq3), run_time=2)
        self.wait(3)
        for i in range(len(thrust_bullet_group)):
            self.play(Write(thrust_bullet_group[i]), run_time=1)
            self.wait(1)

        arrow2 = Arrow(
            start=thrust_system_group.get_right(),
            end=supersonic_system_group.get_left(),
            buff=0.1
        )
        self.play(GrowArrow(arrow2), run_time=1)
        self.wait(1)

        self.play(Write(supersonic_subtitle), run_time=1)
        self.wait(1)
        self.play(Write(supersonic_eq1), Write(supersonic_eq2), Write(supersonic_eq3), run_time=2)
        self.wait(3)
        for i in range(len(supersonic_bullet_group)):
            self.play(Write(supersonic_bullet_group[i]), run_time=1)
            self.wait(1)
        
        self.wait(3)
        self.play(Unwrite(original_system_group), Unwrite(thrust_system_group), Unwrite(supersonic_system_group), Unwrite(End_group), FadeOut(arrow1), FadeOut(arrow2), run_time=2)
        self.wait(1)
        thank_you = Tex(r"Thank you for watching!", font_size=60)
        thank_you.shift(UP*2)
        self.play(Write(thank_you), run_time=1)
        self.wait(2)
        self.play(Unwrite(thank_you), run_time=1)


Manim Community v0.18.1